# testQAOASolver

## Imports

In [1]:
from openqaoa import QAOA
from openqaoa import QUBO
from openqaoa.algorithms import QAOAResult
from openqaoa.backends import create_device

import sys
import os
import asyncio
import json
from IPython.display import clear_output


sys.path.append(os.path.abspath(os.path.join('..', 'functions')))
from JRPClassic import JRPClassic
from JRPRandomGenerator import JRPRandomGenerator

## Configurations

In [2]:
jrp_init_configurations = [
    {'num_agents':2 , 'num_vacnJobs':3 , 'control_restrictions':False},
    {'num_agents':4 , 'num_vacnJobs':2 , 'control_restrictions':False},
    {'num_agents':3,'num_vacnJobs':3,'control_restrictions':False}
]

## Functions

In [3]:
def run_qaoa_workflow(jrp):
    '''
    This function runs the QAOA workflow for a particular JRP instance

    Parameters:
        -jrp
            a JRP object

    Return:
    
    -final_standard_solution
        the solution found after QAOA optimization for a standard form of JRP
    -final_standard_gain
        the gain of the standard solution found after QAOA optimization
    -initial_ising_cost
        the expected value/cost when evaluating the problem hamiltonian with the initial variational params.
    -final_ising_cost
        the expected value/cost when evaluating the problem hamiltonian with the optimized variational params.
    -qaoa_result
        the QAOAResult object, from OpenQAOA
        
    '''
    # QAOA parameters
    device = create_device(location='local', name='qiskit.shot_simulator')  
    circuit_params ={
    # TODO p que sea variable, que pueda controlar su valor. Lo mejor seria con todos los parametros hacerlo
    'p': 3,
    'param_type':'standard',
    'init_type':'ramp',
    'mixer_hamiltonian':'x'
    }
    optimizator_params={
    'method' : 'Powell',
    'maxfev':100,
    'tol':0.001,
    'optimization_progress':True,
    'cost_progress':True,
    'parameter_log':True
    }
    
    # create the QUBO in openqaoa
    jrp.to_qubo()
    terms,weights = jrp.to_openqaoa_format()
    terms,weights = QUBO.convert_qubo_to_ising(len(jrp.instance_dict['allBinaryVariables']), terms, weights)
    jrp_ising = QUBO(n=len(jrp.instance_dict['allBinaryVariables']),terms=terms,weights=weights)
    
    # create QAOA solver and configurations
    qaoa = QAOA()
    #device = create_device(location='local', name='qiskit.shot_simulator')
    qaoa.set_device(device)
    qaoa.set_circuit_properties(**circuit_params)
    qaoa.set_backend_properties(n_shots=10000)
    qaoa.set_classical_optimizer(**optimizator_params)
    
    #compile
    qaoa.compile(jrp_ising)

    # get the initial ising cost, using the initial variational parameters
    initial_ising_cost = qaoa.evaluate_circuit(qaoa.variate_params.asdict())['cost']

    # do the QAOA optimization
    qaoa.optimize()
    qaoa_result = qaoa.result
    
    # get the final ising cost, using the optimized variational parameters
    final_ising_cost = qaoa_result.optimized['cost']

    # create a new QAOA for sampling bitstrings using the optimized variational parameters
    qaoa = QAOA()
    qaoa.set_device(device)
    qaoa.set_circuit_properties(**circuit_params)
    qaoa.set_backend_properties(n_shots=20)
    qaoa.compile(jrp_ising)
    preliminary_qubo_solutions = qaoa.evaluate_circuit(qaoa_result.optimized['angles'])['measurement_results']

    # only takes the feasible solutions
    final_qubo_solutions = []
    for solution in preliminary_qubo_solutions.keys():
        if jrp.is_qubo_feasible(solution):
            final_qubo_solutions.append(solution)
    #print(final_qubo_solutions)

    # translate the solutions from QUBO format to standard format and calculate its gain.
    # store the pair (standardSolution,gain) with the maximal gain
    final_standard_gain = float('-inf')
    final_standard_solution = [-2 for i in range(jrp.instance_dict['num_agents'])]
    for qubo_solution in final_qubo_solutions:
        standard_solution = jrp.quboSolution_to_standardSolution(qubo_solution)
        standard_gain = jrp.calculate_standard_gain(standard_solution)
        
        if final_standard_gain < standard_gain:
            final_standard_solution = standard_solution
            final_standard_gain = standard_gain
    
    return final_standard_solution,final_standard_gain,initial_ising_cost,final_ising_cost,qaoa_result

In [4]:
def run_workflow(jrp):  
    '''
    This function runs A 'testQAOAsolver' workflow for a particular JRP instance

    Parameters:
        -jrp
            a JRP object

    Return:
        -approximation_ratio
            a ratio that shows now near is the QAOA-optimized solution to the actual optimal solution
        -standard_gain_difference
            an absolute difference between the initial standard gain and the final standard gain
        -ising_cost_difference
            an absolute differente between the initial ising cost and the final ising cost
        -opt_standard_solution
            - the optimal solution for JRP in standard form
        -final_standard_solution
            the final solution (QAOA-optimized) solution in standard form
        -qaoa_result
            a QAOAResult object, from OpenQAOA
    '''
    # initial configuration
    initial_standard_solution = [-1 for i in range(jrp.instance_dict['num_agents'])]
    initial_standard_gain = jrp.calculate_standard_gain(initial_standard_solution)

    #optimal configuration
    opt_standard_solution,opt_standard_gain = jrp.solve_standard_with_bruteforce(debug_every=0)
    
    # final configuration - after QAOA optimization
    final_standard_solution,final_standard_gain,initial_ising_cost,final_ising_cost,qaoa_result = run_qaoa_workflow(jrp)

    if opt_standard_gain == 0:
        approximation_ratio = None
    else:
        approximation_ratio = final_standard_gain / opt_standard_gain
    standard_gain_difference = final_standard_gain - initial_standard_gain
    ising_cost_difference = final_ising_cost - initial_ising_cost
    
    # TODO convegence curve 

    return approximation_ratio,standard_gain_difference,ising_cost_difference,opt_standard_solution,final_standard_solution,qaoa_result

    
#sample_workflows(jrp_init_configurations[1],'0',1)

In [7]:
def sample_workflows(jrp_init_configuration,configuration_name,sample_size):
    '''
    this function do a sample of 'testQAOAsolver' workflows and save them in json files

    Parameters:
        jrp_init_configuration
            a dictionary with the parameters for the JRPRandomGenerator
        configuration_name
            a string with the name of the configuration, so as to identify the json file
        sample_size            
    '''
    # create the random JRP instances generator
    jrp_gen = JRPRandomGenerator(**jrp_init_configuration)

    # starts the samplins
    samples = {'configuration':jrp_init_configuration}
    for sample_index in range(sample_size):
        #create a random instance
        jrp_instance_dict = jrp_gen.generate_random_instance()
        jrp_instance_dict = json.loads(jrp_instance_dict)
        #print(jrp_instance_dict)
        jrp = JRPClassic(jrp_instance_dict)
        
        # get the important results
        print('running sample ',sample_index)
        approximation_ratio,standard_gain_difference, ising_cost_difference,opt_standard_solution, final_standard_solution,qaoa_result = run_workflow(jrp)

        # creates the sample dats structure and saves it
        sample = {
            'instance': jrp_instance_dict,
            'approximation_ratio':approximation_ratio,
            'standard_gain_difference':standard_gain_difference,
            'ising_cost_difference':ising_cost_difference,
            'opt_standard_solution':opt_standard_solution,
            'final_standard_solution':final_standard_solution,
            'result':qaoa_result.asdict()
        }
        samples[sample_index] = sample
        with open('./conf%s.json'%(str(configuration_name)), 'w', encoding='utf-8') as file:
            json.dump(samples, file, ensure_ascii=False, indent=4)

## Experiment

In [8]:
sample_size = 5
for itr,conf in enumerate(jrp_init_configurations):
    print('CONFIGURATION ',itr)
    sample_workflows(conf,str(itr),sample_size)
    clear_output(wait=True)


CONFIGURATION  0
running sample  0
running sample  1
running sample  2



KeyboardInterrupt

